# Fase 6: Resolución numérica de la Ecuación diferencial de Black-Scholes-Merton

Implementar un esquema de diferencias finitas implícitas para resolver la ecuación
diferencial de Black-Scholes-Merton, en el caso de una PUT europea y una opción binaria.

In [39]:
#Importo mis librerias
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_banded
from scipy.sparse import diags
import Funciones

### Parámetros iniciales

In [40]:
# Defino los parámetros iniciales
S0 = 112      # Precio inicial del activo
K = 115       # Precio de ejercicio
r = 0.08      # Tasa libre de riesgo
sigma = 0.18  # Volatilidad
T = 1         # Tiempo a vencimiento (en años)

## Discretización del tiempo y riqueza

Definimos los parámetros de la malla y posteriormente generamos la malla de tiempo y riqueza.

\begin{equation*}
\Delta t = \frac{T}{N}, \quad \Delta S = \frac{S_{max}}{M}
\end{equation*}

In [41]:
#Parámetros de la malla
S_max= 2*K  # Precio máximo del activo
M= 100  # Número de puntos en la malla de precios
N = 100  # Número de puntos en la malla de tiempo
dS=S_max/M  # Tamaño del paso en la malla de precios
dT = T/N  # Tamaño del paso en la malla de tiempo

#Vectores de precios y tiempos
S = np.linspace(0, S_max, M+1)  # Vector de precios
t = np.linspace(0, T, N+1)  # Vector de tiempos

## Implementar condiciones de frontera apropiadas

Generamos la malla de tiempo y riqueza, y luego definimos la condición de frontera al vencimiento de la opción PUT europea que es:
\begin{equation*}
V(S, T) = \max(K - S, 0)   
\end{equation*}
Y para la opción binaria:
\begin{equation*}
V(S, T) = \begin{cases}
1 & \text{si } S < K \\
0 & \text{si } S \geq K
\end{cases}
\end{equation*}

Además definimos las condiciones de frontera para $S = 0$ y $S = S_{max}$:
\begin{equation*}
V(0, t) = K e^{-r(T-t)}, \quad V(S_{max}, t) = 0
\end{equation*}

In [42]:
V = np.zeros((M+1, N+1))  # Malla de precios y tiempos
V[:, -1] = np.maximum(K-S, 0)  # Condición de frontera al vencimiento
#Condiciones de frontera para cada t
V[0, :] = K * np.exp(-r * (T - np.linspace(0, T, N + 1)))   # V(0, t)
V[-1, :] = 0                                                # V(S_max, t)

## Solucionar la ecuación diferencial parcial

Implementamos el esquema de diferencias finitas implícitas para resolver la ecuación diferencial de Black-Scholes-Merton. Definimos los coeficientes del sistema de la siguiente manera:
\begin{equation*}
\alpha= \Delta t \left( \frac{a}{\Delta S^2} - \frac{b}{2 \Delta S} \right), \quad
\beta = 1 - \Delta t \left( \frac{a}{\Delta S^2} + c \right), \quad
\gamma = \Delta t \left( \frac{a}{\Delta S^2} + \frac{b}{2 \Delta S} \right)
\end{equation*}

Donde $a = \frac{1}{2} \sigma^2 S^2$, $b = r S$, y $c = r$.

Posterior a esto, construimos la matriz del sistema y resolvemos el sistema lineal en cada paso de tiempo hacia atrás.

In [43]:
#Definimos los coeficientes del sistema lineal
a=0.5* sigma**2 * S[1:-1]**2
b=r*S[1:-1]
c=r

alpha=dT*(a/dS**2 - b/(2*dS))
beta=1-dT*(2*a/dS**2 + c)
gamma=dT*(a/dS**2 + b/(2*dS))

In [44]:
#Matriz tridiagonal
A = diags([alpha[1:] ,beta ,gamma[:-1]],offsets=[-1,0,1],format='csr')

In [ ]:
#Calculamos la solución hacia atrás en el tiempo
put_sol, _= Funciones.sol_put(M, V, alpha,K, gamma, A, S0, S,N)

In [ ]:
bin_sol,_ = Funciones.sol_put(M, V, alpha,K, gamma, A, S0, S,N, tipo='binario')

## Calcular el error

Calcular el error en términos del tamaño de las subdiviciones de $\Delta t$ y $\Delta S$ de la
discretización.

In [47]:
valoresMN=[(10,10), (20,20), (50,50), (100,100), (200,200), (500,500), (1000,1000)]
errores_put = Funciones.calcular_errores(S0, K, r, sigma, T, valoresMN,V,alpha,gamma, A, S, 'put')
errores_binario = Funciones.calcular_errores(S0, K, r, sigma, T, valoresMN,V,alpha,gamma, A, S, 'binario')

for M, N, num, ref, err in errores_put:
    print(f"M={M}, N={N} -> Valor numérico: {num:.4f}, Valor exacto: {ref:.4f}, Error: {err:.4e}")

AttributeError: module 'Funciones' has no attribute 'calcular_errores'